### спасибо вот этому человеку за часть кода / thanks to this person for some of the code
https://blog.csdn.net/weixin_43330946/article/details/89501865

In [ ]:
import nibabel as nib
import numpy as np
import imageio
import os
import re
import shutil
import pandas as pd
from PIL import Image

In [ ]:
df = pd.read_csv('../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/survival_info.csv')
print(df[df['Brats20ID']=='BraTS20_Training_355'])
df = df.drop(index=[225]).reset_index(drop=True)
df.head()

In [ ]:
# прописываем пути к папкам (файлам) / write paths to folders (files)
for i in range(df.shape[0]):
    row = df.loc[i]
    patient_id = row.Brats20ID
    path = f'../input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/{patient_id}/' #{patient_id}_flair.nii'
    #print(label)
    df.loc[(df['Brats20ID']==f'{patient_id}'),'dirs']=str(path) #['patient_id']

In [ ]:
df.head()

In [ ]:
# конвертация нифти в пнг с сохранением результата / converting nifti to png with saving the result
def read_niifile(niifile):  # Read niifile file
    img = nib.load(niifile)  # Download niifile file (actually extract the file)
    img_fdata = img.get_fdata()  # Get niifile data
    return img_fdata


def save_fig(file, savepicdir):  # Save as picture
    fdata = read_niifile(file)  # Call the above function to get data
    (x, y, z) = fdata.shape  # Get data shape information: (length, width, dimension-Number of slices)
    for k in range(z):
        silce = fdata[k, :, :]  # Three positions represent three slices at different angles
        imageio.imwrite(os.path.join(savepicdir, '{}.png'.format(k)), silce)
        # Save the slice information as png format

In [ ]:
# создаем папки / create folders
if not os.path.isdir('./brats20png'):
    os.mkdir('./brats20png')
for i in range(len(df)):
    directori = df.loc[i]['Brats20ID']
    if not os.path.isdir(f'./brats20png/{directori}'):
        os.mkdir(f'./brats20png/{directori}')
#     if not os.path.isdir(f'./brats20png/{directori}/FLAIR'):
#         os.mkdir(f'./brats20png/{directori}/FLAIR')

In [ ]:
# конвертация flair в png / flair to png conversion
for i in range(df.shape[0]):
    row = df.loc[i]
    patient_id = row.Brats20ID
    path = f'{row.dirs}{patient_id}_flair.nii'
    savedir = f'./brats20png/{patient_id}'
    img = save_fig(path, savedir)
    del img

In [ ]:
# создаем папки для масок / create folders for masks
if not os.path.isdir('./brats20masks'):
    os.mkdir('./brats20masks')
for i in range(len(df)):
    directori = df.loc[i]['Brats20ID']
    if not os.path.isdir(f'./brats20masks/{directori}'):
        os.mkdir(f'./brats20masks/{directori}')
#     if not os.path.isdir(f'./brats20masks/{directori}/FLAIR'):
#         os.mkdir(f'./brats20masks/{directori}/FLAIR')

In [ ]:
# конвертация seg в png / converting seg to png
for i in range(df.shape[0]):
    row = df.loc[i]
    patient_id = row.Brats20ID
    path = f'{row.dirs}{patient_id}_seg.nii'
    savedir = f'./brats20masks/{patient_id}'
    img = save_fig(path, savedir)
    del img

In [ ]:
# удаляем маски и изображения, если количество пикселей маски меньше 600 / remove masks and images if the number of mask pixels is less than 600
for i in range(df.shape[0]):
    row = df.loc[i]
    patient_id = row.Brats20ID
    path = f'./brats20masks/{patient_id}'
    impath = f'./brats20masks/{patient_id}'
    for file in os.listdir(path):
        im=np.array(Image.open(f'./brats20masks/{patient_id}/{file}').convert('RGB'))
        result = np.count_nonzero(np.all(im==64,axis=2))
        result1 = np.count_nonzero(np.all(im==127,axis=2))
        result2 = np.count_nonzero(np.all(im==255,axis=2))
        result3 = result + result1 + result2
        if result3<600:
            os.remove(f'./brats20masks/{patient_id}/{file}')
            os.remove(f'./brats20png/{patient_id}/{file}')

In [ ]:
# копируем изображения и маски в 1 папку (если есть желание) / copy images and masks into 1 folder (if you wish)
if not os.path.isdir('./bratsimg'):
    os.mkdir('./bratsimg')
if not os.path.isdir('./bratsimg/img_brats'):
    os.mkdir('./bratsimg/img_brats')
if not os.path.isdir('./bratsimg/mask_brats'):
    os.mkdir('./bratsimg/mask_brats')
a = 0
b = 0
for i in range(df.shape[0]):
    row = df.loc[i]
    patient_id = row.Brats20ID
    path = f'./brats20masks/{patient_id}'
    impath = f'./brats20png/{patient_id}'
    img_files=sorted(os.listdir(f'{impath}'), key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    mask_files=sorted(os.listdir(f'{path}'), key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    for file, mask in zip(img_files, mask_files):
        shutil.copyfile(f'{impath}/{file}', f'./bratsimg/img_brats/{a}.png')
        shutil.copyfile(f'{path}/{file}', f'./bratsimg/mask_brats/{b}.png')
        a = a+1
        b = b+1

In [ ]:
!zip -r brats20png.zip /kaggle/working

In [ ]:
from IPython.display import FileLink
FileLink('./brats20png.zip')